# Test d'un modèle de NER générique sur des données différentes de LECTAUREP avec Stanza

On essaye d'appliquer un modèle de NER sur des données de nature différente. Pour cela, on utilise un fichier du corpus du projet DAHN (https://github.com/FloChiff/DAHNProject ; https://digitalintellectuals.hypotheses.org/), plus précisément une lettre écrite par Paul d'Estournelles de Constant. Le type de langage, ici correspondant au genre épistolaire, est totalement différent de ce que l'on peut rencontrer dans les pages des répertoires de notaires. De plus, le texte ne contient pas de bruit comme on peut trouver dans les textes résultants de l'HTR.

On sélectionne la lettre 569 : https://github.com/FloChiff/DAHNProject/blob/master/Correspondence/Paul_d_Estournelles_de_Constant/Corpus/Lettre569_3octobre1919.xml

Après avoir récupéré le contenu de la balise `<text>`, un nettoyage à la main léger (suppression des sauts de lignes au sein des paragraphes, suppression des balises `<del>`) a été effectué pour avoir un texte exploitable.

## Imports 

In [48]:
import csv

from bs4 import BeautifulSoup
import stanza

## Fonction d'ouverture de fichier texte

In [14]:
def open_text_file(filepath):
    with open(filepath, 'r', encoding='UTF-8') as fh:
        sample = fh.read()
        
    return sample

# Ouverture du fichier source et création d'un fichier BIOES de la prédiction NER

In [4]:
dahn_sample = open_text_file('../../corpus_test/dahn/Lettre569_3octobre1919.txt')

In [6]:
print(dahn_sample)

L'inconvénient de mes voyages est que, loin de satisfaire, ils avivent ma curiosité ; ils renouvellent les
sources de mon attachement à tout ce qui vit, et mon ambition d'être utile. Pour être utiles, commençons par sentir
et par comprendre; il est trop commode de ne pas comprendre.Je comprends trop bien mon pays. Je viens d'y faire un grand voyage qui, pour un Américain, serait une
promenade de quelques heures, car je n'ai vu que mon département de la Sarthe, et encore pas tout entier. Grâce à
l'excellente petite auto que je me suis décidé à acheter à New-York et que chacun ici m'envie, je viens d'en faire
le tour, intérieurement, avec tout autant de méthode et de préparation que s'il s'était agi de "couvrir" le
continent américain comme jadis, en 1911, sous vos auspices et par vos soins. Voyage de trois jours et trois nuits pour
parcourir 500 Km, mais quel parcours ! Et quel pays ! J'étais seul avec mon chauffeur, silencieux et réservé ; j'ai
pu'savourer, sans
à Monsieur le Président

In [11]:
nlp = stanza.Pipeline(lang='fr', processors='tokenize, ner')
doc = nlp(dahn_sample)

2021-08-06 22:55:26 WARNING: Language fr package default expects mwt, which has been added
2021-08-06 22:55:26 INFO: Loading these models for language: fr (French):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| ner       | wikiner |

2021-08-06 22:55:26 INFO: Use device: cpu
2021-08-06 22:55:26 INFO: Loading: tokenize
2021-08-06 22:55:26 INFO: Loading: mwt
2021-08-06 22:55:26 INFO: Loading: ner
2021-08-06 22:55:27 INFO: Done loading processors!


In [99]:
with open("out/Lettre569_3octobre1919_ner_prediction.bioes", "w+", encoding="UTF-8") as fh:
    for sent in doc.sentences:
        for token in sent.tokens:
            BIO_line = token.text + '\t' + token.ner
            fh.write(BIO_line)
            fh.write('\n')

## Visualisation de la reconnaissance d'entités nommées selon le format BIOES

In [92]:
print(*[f'token: {token.text}\tner: {token.ner}' for sent in doc.sentences for token in sent.tokens], sep='\n')

token: L'	ner: O
token: inconvénient	ner: O
token: de	ner: O
token: mes	ner: O
token: voyages	ner: O
token: est	ner: O
token: que	ner: O
token: ,	ner: O
token: loin	ner: O
token: de	ner: O
token: satisfaire	ner: O
token: ,	ner: O
token: ils	ner: O
token: avivent	ner: O
token: ma	ner: O
token: curiosité	ner: O
token: ;	ner: O
token: ils	ner: O
token: renouvellent	ner: O
token: les	ner: O
token: sources	ner: O
token: de	ner: O
token: mon	ner: O
token: attachement	ner: O
token: à	ner: O
token: tout	ner: O
token: ce	ner: O
token: qui	ner: O
token: vit	ner: O
token: ,	ner: O
token: et	ner: O
token: mon	ner: O
token: ambition	ner: O
token: d'	ner: O
token: être	ner: O
token: utile	ner: O
token: .	ner: O
token: Pour	ner: O
token: être	ner: O
token: utiles	ner: O
token: ,	ner: O
token: commençons	ner: O
token: par	ner: O
token: sentir	ner: O
token: et	ner: O
token: par	ner: O
token: comprendre	ner: O
token: ;	ner: O
token: il	ner: O
token: est	ner: O
token: trop	ner: O
token: commode	ner: O
to

Cette expérience n'a malheureusement pas pu être menée jusqu'à son terme, des problèmes ayant été rencontrées lors de l'évaluation de la prédiction NER avec une vérité de terrain NER produite avec la plateforme d'annotation Inception. Nous nous contentons donc d'une simple visualisation.

Des problèmes ont été rencontrées dans l'utilisation de NERVAL que je n'ai pas su résoudre. 

On effectue donc à nouveau le même procédé expérimental avec spaCy.